In [1]:
!pip install confluent-kafka==1.5

In [2]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
from confluent_kafka import KafkaException
import sys
from uuid import uuid4

In [3]:
bootstrap_server = "kafka:9092" # Brokers act as cluster entripoints

In [4]:
conf = {'bootstrap.servers': bootstrap_server}

In [5]:
a = AdminClient(conf)

In [6]:
md = a.list_topics(timeout=10)
print(" {} topics:".format(len(md.topics)))
for t in iter(md.topics.values()):
    if t.error is not None:
        errstr = ": {}".format(t.error)
    else:
        errstr = ""
    ##if not (str(t)).startswith("_"):
    print("  \"{}\" with {} partition(s){}".format(t, len(t.partitions), errstr))

 4 topics:
  "Temperature_Humidity_SensorEvent" with 1 partition(s)
  "TemperatureHumiditySensorEvent" with 1 partition(s)
  "_schemas" with 1 partition(s)
  "__consumer_offsets" with 50 partition(s)


In [7]:
#fs = a.create_topics([NewTopic("test1p", num_partitions=1, replication_factor=1)])

#for topic, f in fs.items():
#    try:
#        f.result()  # The result itself is None
#        print("Topic {} created".format(topic))
#    except Exception as e:
#        print("Failed to create topic {}: {}".format(topic, e))

In [8]:
from confluent_kafka import SerializingProducer
from confluent_kafka.serialization import *
import time

topic = "Temperature_Humidity_Sensor_Event"

def delivery_report(err, msg):
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        print("Produced record to topic {} partition [{}] @ offset {}"
              .format(msg.topic(), msg.partition(), msg.offset()))

In [9]:
producer_conf = {
        'bootstrap.servers': bootstrap_server,
        'key.serializer': StringSerializer('utf_8'),
        'value.serializer': StringSerializer('utf_8')
}

producer = SerializingProducer(producer_conf)

## run the following cell to start streaming the data

In [10]:
import json
from random import gauss
measurement_dict = {
    "1": {"sensor":"S1", "temperature":290, "humidity":30, "ts":1595332800},
    "2": {"sensor":"S2", "temperature":105, "humidity":55, "ts":1595332815},
    "3": {"sensor":"S2", "temperature":110, "humidity":60, "ts":1595332845},
    "4": {"sensor":"S1", "temperature":305, "humidity":38, "ts":1595332860},
    "5": {"sensor":"S2", "temperature":120, "humidity":65, "ts":1595332875},
    "6": {"sensor":"S2", "temperature":115, "humidity":60, "ts":1595332905},
    "7": {"sensor":"S1", "temperature":280, "humidity":45, "ts":1595332920},
    "8": {"sensor":"S2", "temperature":110, "humidity":67, "ts":1595332935},
    "9": {"sensor":"S2", "temperature":115, "humidity":72, "ts":1595332965},
    "10": {"sensor":"S1", "temperature":280,"humidity":22, "ts":1595332980},
    "11": {"sensor":"S2", "temperature":95, "humidity":65, "ts":1595332995},
    "12": {"sensor":"S2", "temperature":90, "humidity":60,  "ts":1595333025},
    "13": {"sensor":"S1", "temperature":285, "humidity":32, "ts":1595333040},
    "14": {"sensor":"S2", "temperature":100, "humidity":55, "ts":1595333055},
    "15": {"sensor":"S2", "temperature":105, "humidity":60, "ts":1595333085}
}
#stream the original data with the original timestamps
for measurement in measurement_dict.values():
    key = measurement["sensor"]
    value = measurement
    producer.produce(topic=topic, value=json.dumps(value), key=key, on_delivery=delivery_report)
    print(value)
    producer.poll(1)

timestamp = 1595333085
#create new data replacing the timestamp with now
for measurement in measurement_dict.values():
    timestamp = timestamp + 3
    key = measurement["sensor"]
    measurement["ts"] = timestamp
    value = measurement
    producer.produce(topic=topic, value=json.dumps(value), key=key, on_delivery=delivery_report)
    print(value)
    producer.poll(1)
    time.sleep(3)

{'sensor': 'S1', 'temperature': 290, 'humidity': 30, 'ts': 1595332800}
{'sensor': 'S2', 'temperature': 105, 'humidity': 55, 'ts': 1595332815}
Produced record to topic Temperature_Humidity_Sensor_Event partition [0] @ offset 0
Produced record to topic Temperature_Humidity_Sensor_Event partition [0] @ offset 1
{'sensor': 'S2', 'temperature': 110, 'humidity': 60, 'ts': 1595332845}
Produced record to topic Temperature_Humidity_Sensor_Event partition [0] @ offset 2
{'sensor': 'S1', 'temperature': 305, 'humidity': 38, 'ts': 1595332860}
Produced record to topic Temperature_Humidity_Sensor_Event partition [0] @ offset 3
{'sensor': 'S2', 'temperature': 120, 'humidity': 65, 'ts': 1595332875}
Produced record to topic Temperature_Humidity_Sensor_Event partition [0] @ offset 4
{'sensor': 'S2', 'temperature': 115, 'humidity': 60, 'ts': 1595332905}
Produced record to topic Temperature_Humidity_Sensor_Event partition [0] @ offset 5
{'sensor': 'S1', 'temperature': 280, 'humidity': 45, 'ts': 1595332920}

## Run the following cell if you would like the streaming to keep going on

In [12]:
while (True):
    for measurement in measurement_dict.values():
        timestamp = timestamp + 3
        key = measurement["sensor"]
        measurement["ts"] = timestamp
        value = measurement
        producer.produce(topic=topic, value=json.dumps(value), key=key, on_delivery=delivery_report)
        print(value)
        producer.poll(1)
        time.sleep(3)

{'sensor': 'S1', 'temperature': 290, 'humidity': 30, 'ts': 1595333133}
Produced record to topic Temperature_Humidity_Sensor_Event partition [0] @ offset 30
{'sensor': 'S2', 'temperature': 105, 'humidity': 55, 'ts': 1595333136}
Produced record to topic Temperature_Humidity_Sensor_Event partition [0] @ offset 31
{'sensor': 'S2', 'temperature': 110, 'humidity': 60, 'ts': 1595333139}
Produced record to topic Temperature_Humidity_Sensor_Event partition [0] @ offset 32
{'sensor': 'S1', 'temperature': 305, 'humidity': 38, 'ts': 1595333142}
Produced record to topic Temperature_Humidity_Sensor_Event partition [0] @ offset 33
{'sensor': 'S2', 'temperature': 120, 'humidity': 65, 'ts': 1595333145}
Produced record to topic Temperature_Humidity_Sensor_Event partition [0] @ offset 34
{'sensor': 'S2', 'temperature': 115, 'humidity': 60, 'ts': 1595333148}
Produced record to topic Temperature_Humidity_Sensor_Event partition [0] @ offset 35
{'sensor': 'S1', 'temperature': 280, 'humidity': 45, 'ts': 15953

KeyboardInterrupt: 